In [6]:
# -----------------------------------------------------------
# FASE 1 (V3.0): EL GRAN BALANCEO (SOLUCIÓN DEFINITIVA)
# -----------------------------------------------------------
import pandas as pd
import os
import glob
import shutil
import random
import re

# 1. LIMPIEZA TOTAL
dirs = ['dvna', 'NodeGoat', 'juice-shop', 'payloads', 'express', 'lodash', 'react', 'vue', 'jquery']
for d in dirs:
    if os.path.exists(d): shutil.rmtree(d)

print("🚀 DESCARGANDO MONTAÑAS DE CÓDIGO...")

# A) FUENTES VULNERABLES (Payloads + Apps)
!git clone https://github.com/juice-shop/juice-shop.git --depth 1 -q
!git clone https://github.com/payloadbox/xss-payload-list.git ./payloads/xss -q
!git clone https://github.com/payloadbox/sql-injection-payload-list.git ./payloads/sqli -q

# B) FUENTES SEGURAS (NECESITAMOS MUCHAS MÁS)
print("⬇️ Descargando el ecosistema JS seguro (React, Vue, JQuery, D3)...")
!git clone https://github.com/facebook/react.git --depth 1 -q
!git clone https://github.com/vuejs/core.git ./vue --depth 1 -q
!git clone https://github.com/jquery/jquery.git --depth 1 -q
!git clone https://github.com/d3/d3.git --depth 1 -q
!git clone https://github.com/expressjs/express.git --depth 1 -q

# -----------------------------------------------------------
# MOTOR DE PROCESAMIENTO
# -----------------------------------------------------------
data = []

# 1. PROCESAR ATAQUES (VULNERABLES)
print("⚙️ Generando vectores de ataque...")
# Payloads puros inyectados en contexto JS
payload_files = glob.glob("./payloads/**/*.txt", recursive=True)
for f in payload_files:
    try:
        with open(f, 'r', errors='ignore') as file:
            lines = file.readlines()
            for line in lines:
                line = line.strip()
                if 5 < len(line) < 200: # Filtro de longitud
                    # Contexto 1: Variable directa
                    data.append({'code': f'var s = "{line}";', 'label': 1})
                    # Contexto 2: Función peligrosa
                    data.append({'code': f'eval("{line}");', 'label': 1})
    except: pass

# Juice Shop (Código real vulnerable)
js_files = glob.glob("./juice-shop/**/*.js", recursive=True) + glob.glob("./juice-shop/**/*.ts", recursive=True)
for f in js_files:
    if "test" in f or "node_modules" in f: continue
    try:
        with open(f, 'r', errors='ignore') as file:
            content = file.read()
            if len(content) > 30:
                data.append({'code': content, 'label': 1})
    except: pass

# 2. PROCESAR CÓDIGO SEGURO (MASSIVE SCRAPING)
print("⚙️ Recolectando código seguro...")
safe_dirs = ['react', 'vue', 'jquery', 'd3', 'express']
for d in safe_dirs:
    # Buscamos JS y TS
    files = glob.glob(f"./{d}/**/*.js", recursive=True) + glob.glob(f"./{d}/**/*.ts", recursive=True)
    print(f"   - Escaneando {d}: {len(files)} archivos.")

    for f in files:
        if "test" in f or "node_modules" in f or "dist" in f: continue
        try:
            with open(f, 'r', errors='ignore') as file:
                content = file.read()
                # Bajamos el filtro a 20 caracteres para captar funciones pequeñas utilitarias
                if len(content) > 20:
                    data.append({'code': content, 'label': 0})
        except: pass

# -----------------------------------------------------------
# BALANCEO FORZADO (TARGET: 10,000 DATOS)
# -----------------------------------------------------------
df = pd.DataFrame(data)
vuln = df[df['label']==1]
safe = df[df['label']==0]

print(f"\n📊 Estadísticas Pre-Balanceo:")
print(f"   - Vulnerables: {len(vuln)}")
print(f"   - Seguros:     {len(safe)}")

# ESTRATEGIA: Definimos un tamaño objetivo.
# Si hay 16k vulnerables, forzamos tener al menos 8k seguros clonándolos.
TARGET_SIZE = 8000

print(f"⚖️ Ajustando dataset a ~{TARGET_SIZE * 2} ejemplos totales...")

# Upsampling / Downsampling
df_vuln_bal = vuln.sample(n=TARGET_SIZE, replace=True, random_state=42)
df_safe_bal = safe.sample(n=TARGET_SIZE, replace=True, random_state=42)

final_df = pd.concat([df_vuln_bal, df_safe_bal]).sample(frac=1).reset_index(drop=True)

print(f"\n✅ DATASET MAESTRO LISTO: {len(final_df)} ejemplos.")
print(final_df['label'].value_counts())

final_df.to_csv('dataset_pro_v3.csv', index=False)

🚀 DESCARGANDO MONTAÑAS DE CÓDIGO...
⬇️ Descargando el ecosistema JS seguro (React, Vue, JQuery, D3)...
⚙️ Generando vectores de ataque...
⚙️ Recolectando código seguro...
   - Escaneando react: 4267 archivos.
   - Escaneando vue: 512 archivos.
   - Escaneando jquery: 225 archivos.
   - Escaneando d3: 11 archivos.
   - Escaneando express: 142 archivos.

📊 Estadísticas Pre-Balanceo:
   - Vulnerables: 17033
   - Seguros:     2400
⚖️ Ajustando dataset a ~16000 ejemplos totales...

✅ DATASET MAESTRO LISTO: 16000 ejemplos.
label
1    8000
0    8000
Name: count, dtype: int64


In [7]:
# -----------------------------------------------------------
# FASE 2: PREPROCESAMIENTO Y TOKENIZACIÓN
# -----------------------------------------------------------

# 1. Instalamos la librería 'transformers' de Hugging Face
# Esta es la herramienta estándar mundial para IA de texto/código
!pip install transformers torch -q

from transformers import AutoTokenizer
import torch
from sklearn.model_selection import train_test_split

# -----------------------------------------------------------
# CARGA DEL TOKENIZER (EL TRADUCTOR)
# -----------------------------------------------------------
# Usaremos "microsoft/codebert-base".
# Es un modelo que Microsoft pre-entrenó con MILLONES de repositorios.
# Ya "sabe" JavaScript, solo le enseñaremos seguridad.
print("⬇️ Descargando el 'Cerebro' base (CodeBERT)...")
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

# -----------------------------------------------------------
# FUNCIÓN DE TRADUCCIÓN
# -----------------------------------------------------------
def preprocess_function(examples):
    # Toma el texto del código
    inputs = examples

    # Tokeniza:
    # - padding='max_length': Rellena con ceros hasta llegar al tope.
    # - truncation=True: Corta si es muy largo (CodeBERT suele aceptar hasta 512 tokens).
    # - return_tensors="pt": Devuelve tensores de PyTorch (formato matemático).
    return tokenizer(
        inputs,
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

print("⚙️ Traduciendo código a números...")

# Extraemos solo las listas de códigos y etiquetas del DataFrame anterior
codes = final_df['code'].tolist()
labels = final_df['label'].tolist()

# ⚠️ División Clave: Entrenamiento vs Prueba
# Para que el modelo sea PRECISO y no tenga falsos positivos,
# debemos guardar una parte de los datos (20%) que el modelo NUNCA verá
# durante el entrenamiento. Solo los usaremos al final para el examen.
train_codes, val_codes, train_labels, val_labels = train_test_split(
    codes, labels, test_size=0.2, random_state=42
)

# Aplicamos la tokenización
train_encodings = preprocess_function(train_codes)
val_encodings = preprocess_function(val_codes)

print("✅ Tokenización completada.")

# -----------------------------------------------------------
# VISUALIZACIÓN DE LO QUE VE LA IA
# -----------------------------------------------------------
print("\n🔍 ASÍ VE EL CÓDIGO LA IA (Primer ejemplo):")
print("Texto original:", train_codes[0][:50], "...")
print("Vector Numérico (Input IDs):", train_encodings['input_ids'][0][:20])
print("Máscara de Atención (Attention Mask):", train_encodings['attention_mask'][0][:20])

# PROFESOR: Explicación de la salida
# 'Input IDs': Son las palabras convertidas a números.
# 'Attention Mask': Son 1s y 0s. Los 1s son código real, los 0s son relleno.
# Esto le dice al modelo "Ignora los ceros, concéntrate en los unos".


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


⬇️ Descargando el 'Cerebro' base (CodeBERT)...
⚙️ Traduciendo código a números...
✅ Tokenización completada.

🔍 ASÍ VE EL CÓDIGO LA IA (Primer ejemplo):
Texto original: eval("<html draggable="true" ondragenter="alert(1) ...
Vector Numérico (Input IDs): tensor([    0, 28017, 46469, 41552,  6660,  8386,   571,   868, 40635, 29225,
          113,    15, 10232,  1073, 11798, 40635, 39383,  1640,   134,    43])
Máscara de Atención (Attention Mask): tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [9]:
# -----------------------------------------------------------
# FASE 3: CREACIÓN DEL DATASET Y CARGA DEL MODELO
# -----------------------------------------------------------

import torch
from transformers import AutoModelForSequenceClassification

# 1. DEFINIMOS LA CLASE DATASET
# Esta clase es el "puente" estandarizado entre tus datos y PyTorch
class SecurityDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        # Esta función se ejecuta cada vez que el modelo pide un dato
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        # Le dice al modelo cuántos datos totales tenemos
        return len(self.labels)

# 2. INSTANCIAMOS LOS DATASETS
# Convertimos los datos de la Fase 2 en objetos Dataset
train_dataset = SecurityDataset(train_encodings, train_labels)
val_dataset = SecurityDataset(val_encodings, val_labels)

print("✅ Datasets de entrenamiento y validación listos.")

# -----------------------------------------------------------
# 3. CARGA DEL MODELO DE CLASIFICACIÓN
# -----------------------------------------------------------
# Aquí ocurre la magia.
# Usamos 'AutoModelForSequenceClassification'.
# Esto toma el cerebro de CodeBERT y le agrega una "cabeza" nueva
# diseñada especificamente para decir SI/NO (num_labels=2).

print("⬇️ Descargando arquitectura de clasificación...")

model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/codebert-base",
    num_labels=2
)

# Movemos el modelo a la GPU si está disponible (para que sea rápido)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

print(f"🚀 Modelo cargado en: {device}")
print("Nota: Si dice 'cpu', el entrenamiento será lento. En Colab, ve a 'Entorno de ejecución' > 'Cambiar tipo' > T4 GPU para acelerarlo.")

✅ Datasets de entrenamiento y validación listos.
⬇️ Descargando arquitectura de clasificación...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🚀 Modelo cargado en: cuda
Nota: Si dice 'cpu', el entrenamiento será lento. En Colab, ve a 'Entorno de ejecución' > 'Cambiar tipo' > T4 GPU para acelerarlo.


In [13]:
# Instalar librería de aceleración obligatoria para PyTorch
!pip install accelerate>=0.26.0 --upgrade
!pip install transformers[torch] --upgrade


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# -----------------------------------------------------------
# FASE 4: ENTRENAMIENTO (CORREGIDO)
# -----------------------------------------------------------

from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# 1. DEFINIMOS LAS MÉTRICAS
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # average='binary' es clave para problemas de SI/NO
    # zero_division=0 evita errores si al principio no predice nada bien
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary', zero_division=0)
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# 2. CONFIGURACIÓN DEL ENTRENAMIENTO
# Nota del Profesor: Hemos cambiado 'evaluation_strategy' por 'eval_strategy'
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,              # Subimos un poco las épocas para asegurar aprendizaje
    per_device_train_batch_size=4,   # Bajamos el batch a 4 por si acaso la memoria es justa
    per_device_eval_batch_size=4,
    warmup_steps=10,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",           # <--- AQUÍ ESTABA EL ERROR (Nombre actualizado)
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,              # Solo guarda los 2 mejores modelos para no llenar el disco
)

# 3. INICIALIZAR EL ENTRENADOR
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("🔥 INICIANDO ENTRENAMIENTO... (Ahora sí)")
trainer.train()

🔥 INICIANDO ENTRENAMIENTO... (Ahora sí)


C:\Users\VICTUS\AppData\Local\Temp\ipykernel_6216\3683666337.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.000000,0.010050,0.998750,0.998720,0.999360,0.998082
2,0.000000,0.008350,0.999062,0.999040,1.000000,0.998082
3,0.000200,0.013591,0.998437,0.998399,1.000000,0.996803
4,0.000200,0.010082,0.999062,0.999040,1.000000,0.998082
5,0.000100,0.007045,0.999062,0.999040,1.000000,0.998082


C:\Users\VICTUS\AppData\Local\Temp\ipykernel_6216\3683666337.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\VICTUS\AppData\Local\Temp\ipykernel_6216\3683666337.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
C:\Users\VICTUS\AppData\Local\Temp\ipykernel_6216\3683666337.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val i

TrainOutput(global_step=16000, training_loss=0.027882219314575196, metrics={'train_runtime': 3354.9359, 'train_samples_per_second': 19.076, 'train_steps_per_second': 4.769, 'total_flos': 1.683910754304e+16, 'train_loss': 0.027882219314575196, 'epoch': 5.0})

In [11]:
# -----------------------------------------------------------
# FASE 5: GUARDADO DEL MODELO (PERSISTENCIA)
# -----------------------------------------------------------
import os

# Nombre de la carpeta donde vivirá tu IA
output_dir = "./mi_modelo_final_gpu"

# Guardamos el modelo (pesos matemáticos) y el tokenizador (diccionario)
print(f"💾 Guardando el modelo en '{output_dir}'...")
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

print("✅ ¡Modelo guardado! Ahora puedes usarlo cuando quieras sin re-entrenar.")

💾 Guardando el modelo en './mi_modelo_final_gpu'...
✅ ¡Modelo guardado! Ahora puedes usarlo cuando quieras sin re-entrenar.


In [1]:
# -----------------------------------------------------------
# FASE 6: EL ESCÁNER (INFERENCIA)
# -----------------------------------------------------------
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import os

# 1. CONFIGURACIÓN
# Pon aquí la ruta de tu carpeta con los 2 archivos
carpeta_objetivo = r"C:\Users\VICTUS\Documents\ESPE\MODELO_v2\examples"  # <--- CAMBIA ESTO (Usa r"" para rutas de Windows)
modelo_path = "./mi_modelo_final_gpu"

# 2. CARGAMOS EL MODELO GUARDADO
print("🔄 Cargando tu IA entrenada...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(modelo_path)
model = AutoModelForSequenceClassification.from_pretrained(modelo_path)
model.to(device)
model.eval() # Modo evaluación (apaga el aprendizaje)

# 3. FUNCIÓN DE PREDICCIÓN
def analizar_archivo(ruta):
    try:
        with open(ruta, 'r', encoding='utf-8', errors='ignore') as f:
            codigo = f.read()
        
        # Filtro básico: Si está vacío, ignorar
        if len(codigo.strip()) < 10: return
        
        # Preparar para la GPU
        inputs = tokenizer(
            codigo, 
            return_tensors="pt", 
            truncation=True, 
            max_length=512, 
            padding=True
        ).to(device)

        # Preguntar al modelo
        with torch.no_grad():
            outputs = model(**inputs)
        
        # Matemáticas finales (Logits -> Porcentajes)
        probs = F.softmax(outputs.logits, dim=-1)
        prediccion = torch.argmax(probs, dim=-1).item()
        confianza = probs[0][prediccion].item() * 100
        
        # RESULTADOS
        nombre_archivo = os.path.basename(ruta)
        print("-" * 60)
        print(f"📄 Archivo: {nombre_archivo}")
        
        if prediccion == 1: # 1 suele ser VULNERABLE en nuestro dataset
            print(f"🚨 ESTADO: \033[91mPELIGROSO / VULNERABLE\033[0m") # Rojo
            print(f"🤖 Confianza IA: {confianza:.2f}%")
            print("⚠️  Se detectaron patrones inseguros.")
        else:
            print(f"✅ ESTADO: \033[92mSEGURO\033[0m") # Verde
            print(f"🛡️ Confianza IA: {confianza:.2f}%")
            
    except Exception as e:
        print(f"Error leyendo {ruta}: {e}")

# 4. EJECUTAR ANÁLISIS
print(f"\n🚀 ESCANEANDO ARCHIVOS EN: {carpeta_objetivo}\n")

if os.path.exists(carpeta_objetivo):
    archivos = [f for f in os.listdir(carpeta_objetivo) if f.endswith(('.js', '.ts', '.py', '.txt'))]
    
    if not archivos:
        print("❌ No encontré archivos .js, .ts o .txt en esa carpeta.")
    
    for archivo in archivos:
        ruta_completa = os.path.join(carpeta_objetivo, archivo)
        analizar_archivo(ruta_completa)
else:
    print("❌ La ruta de la carpeta no existe. Revisa las comillas o barras.")

c:\Users\VICTUS\Documents\ESPE\MODELO_v2\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔄 Cargando tu IA entrenada...

🚀 ESCANEANDO ARCHIVOS EN: C:\Users\VICTUS\Documents\ESPE\MODELO_v2\examples

------------------------------------------------------------
📄 Archivo: code.js
✅ ESTADO: SEGURO
🛡️ Confianza IA: 99.34%
